In [1]:
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,hp,fmin,Trials,tpe
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import mlflow
from mlflow.models import infer_signature
import keras

c:\Users\tkatre\Desktop\mlflow\venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=';')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
#split the data into training,validation and test sets
train,test = train_test_split(data,train_size=0.25,random_state=42)

In [4]:
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()
#test dataset (for hiding from the model)
test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel()

# splitting this train data into train and validation
train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature = infer_signature(train_x,train_y)

In [5]:
#ANN model
def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    # Define model architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean,variance=var),
        keras.layers.Dense(64,activation='relu'),
        keras.layers.Dense(1)
    ])
## compile the model
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params['lr'],momentum=params['momentum']),
                  loss="mean_squared_error",
                  metrics=[keras.metrics.RootMeanSquaredError()])
    
    ##train the ANN model with lr and momentum params with MLFlow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
    # Evaluate the model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)
        eval_rmse = eval_result[1]

        #Log the parameters and result
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        #log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}

In [6]:
def objective(params):
    #MLflow will track the parameters and result for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [7]:
    #setting the parameter
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}


In [8]:
#creating a mlflow experiment
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    #conduct the hyperparameter search using hyperopt
    trials = Trials()
    best =fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    #fetch the details of the best run
    best_run=sorted(trials.results,key=lambda x:x["loss"])[0]

    #log the best parameters,loss,and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    #Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run["loss"]}")

2025/10/29 20:21:31 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/16 ━━━━━━━━━━━━━━━━━━━━ 5s 339ms/step - loss: 28.0128 - root_mean_squared_error: 5.2927
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 15.9429 - root_mean_squared_error: 3.9929 - val_loss: 6.7605 - val_root_mean_squared_error: 2.6001

Epoch 2/3                                            

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 6.4544 - root_mean_squared_error: 2.5406
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.1044 - root_mean_squared_error: 2.0259 - val_loss: 3.4043 - val_root_mean_squared_error: 1.8451

Epoch 3/3                                            

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 3.6915 - root_mean_squared_error: 1.9213
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6392 - root_mean_squared_error: 1.6246 - val_loss: 2.8519 - val_root_mean_squared_error: 1.6887

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.9290 - root_mean_squared_error: 1.3889
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - 

2025/10/29 20:21:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/16 ━━━━━━━━━━━━━━━━━━━━ 4s 277ms/step - loss: 36.1674 - root_mean_squared_error: 6.0139
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6.9231 - root_mean_squared_error: 2.6312 - val_loss: 1.6589 - val_root_mean_squared_error: 1.2880

Epoch 2/3                                                                      

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.8924 - root_mean_squared_error: 1.3756
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0091 - root_mean_squared_error: 1.0045 - val_loss: 0.7914 - val_root_mean_squared_error: 0.8896

Epoch 3/3                                                                      

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.6812 - root_mean_squared_error: 0.8253
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6607 - root_mean_squared_error: 0.8128 - val_loss: 0.6954 - val_root_mean_squared_error: 0.8339

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4

2025/10/29 20:21:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/3                                                                      

 1/16 ━━━━━━━━━━━━━━━━━━━━ 4s 278ms/step - loss: 36.1436 - root_mean_squared_error: 6.0120
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 26.0854 - root_mean_squared_error: 5.1074 - val_loss: 14.6492 - val_root_mean_squared_error: 3.8274

Epoch 2/3                                                                      

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 13.6546 - root_mean_squared_error: 3.6952
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 8.9456 - root_mean_squared_error: 2.9909 - val_loss: 4.8674 - val_root_mean_squared_error: 2.2062

Epoch 3/3                                                                      

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 4.8076 - root_mean_squared_error: 2.1926
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.3766 - root_mean_squared_error: 1.8376 - val_loss: 2.6162 - val_root_mean_squared_error: 1.6175

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 

2025/10/29 20:22:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:40<00:00, 10.10s/trial, best loss: 0.8339017629623413]

2025/10/29 20:22:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.03411187036485888), 'momentum': np.float64(0.7475829530399566)}
Best eval rmse: 0.8339017629623413


In [ ]:
#Inferencing
from mlflow.models import validate_serving_input
model_uri = f"runs:/${mlflow.get_run}"